In [2]:
import os
gpu_num = 0 # 使用 "" 来启用 CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import mitsuba as mi

# Import Sionna RT components
from mysionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

In [3]:
scene = load_scene("Indoor/indoor.xml")

NameError: name 'SolverCoverageMapSensing' is not defined

In [ ]:
#################配置发端天线阵列#################
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

#################配置收端天线阵列#################
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

################创建发射机#########################
tx = Transmitter(name="tx",
                 position=[0,0,2.95])
################ 将发射机加入到场景中##############
scene.add(tx)
#################创建接收机########################
rx = Receiver(name="rx",
              position=[0,0,2.95])
################ 将接收机加入到场景中##############
scene.add(rx)
rx1 = Receiver(name="rx1",
                position=[0,0,0])
scene.add(rx1)
tx.look_at(rx1)
rx.look_at(rx1)
scene.remove('rx1')

In [ ]:
scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
scene.target_names = ["door"]
scene.target_velocities = [(1.,1.,0.)]

In [ ]:
paths = scene.compute_paths(max_depth=4,diffraction=True,scattering=True,edge_diffraction=True)

In [ ]:
v,obj=scene.compute_target_velocities(paths, return_obj_names=True)

In [8]:
subcarrier_spacing = 15e3
num_time_steps = 14
paths.apply_doppler(sampling_frequency=subcarrier_spacing, num_time_steps=num_time_steps,target_velocities=v)